# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Introduction

This program suggests a best place to setup restaurant in the city of Hyderabad, India.

First, this program searches for top destinations in the city. 

In the second step it searches for the restaurants in the city. 

In the third step, it searches for the destination which is having no restaurants in the proximity


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

This project will find an optimum location for opening a restaurant in any city. Hyderabad city of India is taken as example

We will identify all the restaurants in the city and all the famous places to visit in the city

We will create a distance matrix from all the restaurants to the famous places

We will compute the mean distance from restaurants to the famous place

We will identify the famous place with maximum average distance from any restaurant

The identified location will have a famous place but far away from any other restaurant

## Data <a name="data"></a>

We need location data of restraunts and famous places in Hyderabad. Foursquare API is used for this

##### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


##### Define Foursquare Credentials and Version


In [2]:
CLIENT_ID = 'M4KZDPC0BND5E1K0R2OWYC5YAN0MUAUEXX5RTXBUKXI4JXQZ' # your Foursquare ID
CLIENT_SECRET = 'P5FV1CC2PN1GYZWJYEIY5A4G55JLUHZHKWHFQHCX5KRL132M' # your Foursquare Secret
ACCESS_TOKEN = 'B3N25PC1MGJK33IYVH0N1WI0ON4LXQTD3IW1MOLI1E1LWTK4' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M4KZDPC0BND5E1K0R2OWYC5YAN0MUAUEXX5RTXBUKXI4JXQZ
CLIENT_SECRET:P5FV1CC2PN1GYZWJYEIY5A4G55JLUHZHKWHFQHCX5KRL132M


In [33]:
address = 'Hyderabad, Telangana, India'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

17.360589 78.4740613


Set center location to Hyderabad and radius to 10000

Give Category ID in search_query


In [4]:
search_query = '4d4b7104d754a06370d81259'
radius = 10000
print(search_query + ' .... OK!')

4d4b7104d754a06370d81259 .... OK!


#### Define the corresponding URL


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M4KZDPC0BND5E1K0R2OWYC5YAN0MUAUEXX5RTXBUKXI4JXQZ&client_secret=P5FV1CC2PN1GYZWJYEIY5A4G55JLUHZHKWHFQHCX5KRL132M&ll=17.360589,78.4740613&oauth_token=B3N25PC1MGJK33IYVH0N1WI0ON4LXQTD3IW1MOLI1E1LWTK4&v=20180604&categoryId=4d4b7104d754a06370d81259&radius=10000&limit=30'

#### Send the GET Request and examine the results


In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fa6bd11f1355051575014d'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e1d712415207c4cbec540bd',
    'name': 'Charminar',
    'location': {'address': 'Old City',
     'lat': 17.361554582179913,
     'lng': 78.4746766090393,
     'labeledLatLngs': [{'label': 'display',
       'lat': 17.361554582179913,
       'lng': 78.4746766090393}],
     'distance': 125,
     'postalCode': '500002',
     'cc': 'IN',
     'city': 'Hyderabad',
     'state': 'Telangana',
     'country': 'India',
     'formattedAddress': ['Old City', 'Hyderabad 500002', 'Telangana']},
    'categories': [{'id': '4bf58dd8d48988d12d941735',
      'name': 'Monument / Landmark',
      'pluralName': 'Monuments / Landmarks',
      'shortName': 'Landmark',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_monument_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-7-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,4e1d712415207c4cbec540bd,Charminar,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",v-1627024337,False,Old City,17.361555,78.474677,"[{'label': 'display', 'lat': 17.36155458217991...",125,500002,IN,Hyderabad,Telangana,India,"[Old City, Hyderabad 500002, Telangana]",NaN,NaN
1,4cc19bfb67a3b1f77a8dc90e,Chowmahala Palace,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",v-1627024337,False,"Moti Gali, Charminar",17.359300,78.471617,"[{'label': 'display', 'lat': 17.35929974382910...",296,NaN,IN,Hyderabad,Telangana,India,"[Moti Gali, Charminar, Hyderabad, Telangana]",NaN,NaN
2,4ed6fdc1f5b915cfe1c86883,Lal Bahadur Shastri Stadium,"[{'id': '4bf58dd8d48988d184941735', 'name': 'S...",v-1627024337,False,Basheerbagh,17.399140,78.472666,"[{'label': 'display', 'lat': 17.39913964021616...",4293,NaN,IN,Hyderabad,Telangana,India,"[Basheerbagh, Hyderabad, Telangana]",NaN,NaN
3,57dbb159498e24906dcb59ac,Qutb Shahi Heritage Park,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",v-1627024337,False,NaN,17.396019,78.398020,"[{'label': 'display', 'lat': 17.396019, 'lng':...",8989,NaN,IN,NaN,NaN,India,NaN,NaN,NaN
4,5a8c515fe1f0aa07a180302c,Pvr Next Galleria Palyhouse Mall,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",v-1627024337,False,"Opp Punjaguta Metro Station, Srinagar Colony A...",17.429289,78.450195,"[{'label': 'display', 'lat': 17.429289, 'lng':...",8056,500082,IN,Hyderabad,Telangana,India,"[Opp Punjaguta Metro Station, Srinagar Colony ...",Opp Punjaguta Metro Station,NaN


In [60]:
#find size of the data frame
dataframe.shape

(30, 18)

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Charminar,Monument / Landmark,Old City,17.361555,78.474677,"[{'label': 'display', 'lat': 17.36155458217991...",125,500002,IN,Hyderabad,Telangana,India,"[Old City, Hyderabad 500002, Telangana]",NaN,NaN,4e1d712415207c4cbec540bd
1,Chowmahala Palace,History Museum,"Moti Gali, Charminar",17.359300,78.471617,"[{'label': 'display', 'lat': 17.35929974382910...",296,NaN,IN,Hyderabad,Telangana,India,"[Moti Gali, Charminar, Hyderabad, Telangana]",NaN,NaN,4cc19bfb67a3b1f77a8dc90e
2,Lal Bahadur Shastri Stadium,Stadium,Basheerbagh,17.399140,78.472666,"[{'label': 'display', 'lat': 17.39913964021616...",4293,NaN,IN,Hyderabad,Telangana,India,"[Basheerbagh, Hyderabad, Telangana]",NaN,NaN,4ed6fdc1f5b915cfe1c86883
3,Qutb Shahi Heritage Park,History Museum,NaN,17.396019,78.398020,"[{'label': 'display', 'lat': 17.396019, 'lng':...",8989,NaN,IN,NaN,NaN,India,NaN,NaN,NaN,57dbb159498e24906dcb59ac
4,Pvr Next Galleria Palyhouse Mall,Multiplex,"Opp Punjaguta Metro Station, Srinagar Colony A...",17.429289,78.450195,"[{'label': 'display', 'lat': 17.429289, 'lng':...",8056,500082,IN,Hyderabad,Telangana,India,"[Opp Punjaguta Metro Station, Srinagar Colony ...",Opp Punjaguta Metro Station,NaN,5a8c515fe1f0aa07a180302c
5,Pvr Irrum Manzil,Multiplex,NaN,17.420914,78.455236,"[{'label': 'display', 'lat': 17.420914, 'lng':...",7006,500082,IN,Hyderabad,TG,India,"[Hyderabad 500082, TG]",NaN,NaN,5c25af9193bd63002cb7d4e0
6,Uppal stadium,Cricket Ground,Uppal,17.399826,78.537948,"[{'label': 'display', 'lat': 17.39982635522605...",8071,NaN,IN,Hyderabad,Telangana,India,"[Uppal (Stadium), Hyderabad, Telangana]",Stadium,NaN,4f1abfbee4b035290d802952
7,Birla Science Museum,Science Museum,NaN,17.403854,78.469457,"[{'label': 'display', 'lat': 17.40385351664317...",4840,NaN,IN,NaN,Andhra Pradesh,India,[Andhra Pradesh],NaN,NaN,4d048e1a9d33a143c8fcbb78
8,Nizam's Museum,History Museum,"Purani Haveli Road, Purani Haveli, Pathar Gatti",17.366785,78.483000,"[{'label': 'display', 'lat': 17.36678451749927...",1173,50002,IN,Hyderabad,Telangana,India,"[Purani Haveli Road, Purani Haveli, Pathar Gat...",NaN,NaN,516043f1fe706bf2b8e744a8
9,Sudha Cinepolis,Theater,Bela,17.350346,78.474600,"[{'label': 'display', 'lat': 17.350346, 'lng':...",1141,500065,IN,Hyderabad,Telangana,India,"[Bela (Gowlipura), Hyderabad 500065, Telangana]",Gowlipura,NaN,5ae29faf033693002c3f736e


#### Let's visualize the famous places in Hyderabad

In [10]:
dataframe_filtered.name

0                             Charminar
1                     Chowmahala Palace
2           Lal Bahadur Shastri Stadium
3              Qutb Shahi Heritage Park
4      Pvr Next Galleria Palyhouse Mall
5                      Pvr Irrum Manzil
6                         Uppal stadium
7                  Birla Science Museum
8                        Nizam's Museum
9                       Sudha Cinepolis
10    Kotla Vijayabhaskar Reddy Stadium
11          Nampally Exhibition Grounds
12                        Gokul theatre
13                       Galaxy theatre
14                    Terminal Drive-In
15     PVR Hyderabad Next Galleria Mall
16                           KK Gardens
17        Birla Science Center & Museum
18                    Asian Shiva ganga
19                       Konark Theater
20                 Shri Sai Ram Theater
21                  Sree Ramulu Theater
22                        Buddha statue
23                          PVR Cinemas
24                JRC Convention Centre


In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Conrad Hotel

# display all the famous places in the city
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


In [20]:
search_query2 = '4d4b7105d754a06374d81259'
radius2 = 10000
print(search_query2 + ' .... OK!')

4d4b7105d754a06374d81259 .... OK!


In [21]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query2, radius2, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=M4KZDPC0BND5E1K0R2OWYC5YAN0MUAUEXX5RTXBUKXI4JXQZ&client_secret=P5FV1CC2PN1GYZWJYEIY5A4G55JLUHZHKWHFQHCX5KRL132M&ll=17.360589,78.4740613&oauth_token=B3N25PC1MGJK33IYVH0N1WI0ON4LXQTD3IW1MOLI1E1LWTK4&v=20180604&categoryId=4d4b7105d754a06374d81259&radius=10000&limit=30'

In [22]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '60fa6ce4dfdf6825eb023a0e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4f1db31ee4b04ae0856d18ff',
    'name': 'Hotel Nayaab',
    'location': {'address': 'Charminar',
     'lat': 17.368504008597416,
     'lng': 78.47747956353676,
     'labeledLatLngs': [{'label': 'display',
       'lat': 17.368504008597416,
       'lng': 78.47747956353676}],
     'distance': 953,
     'cc': 'IN',
     'country': 'India',
     'formattedAddress': ['Charminar']},
    'categories': [{'id': '4bf58dd8d48988d120951735',
      'name': 'Food Court',
      'pluralName': 'Food Courts',
      'shortName': 'Food Court',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_foodcourt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1627024612',
    'hasPerk': False},
   {'id': '55c3a81a498e744524b8c253',
    'name': 'govind dosas',
    'location': {'address': 'Banj

In [23]:
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2.head()

<ipython-input-23-57dd9101b2b1>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe2 = json_normalize(venues2)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.city,location.state,location.postalCode,location.crossStreet,location.neighborhood
0,4f1db31ee4b04ae0856d18ff,Hotel Nayaab,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1627024612,False,Charminar,17.368504,78.477480,"[{'label': 'display', 'lat': 17.36850400859741...",953,IN,India,[Charminar],NaN,NaN,NaN,NaN,NaN
1,55c3a81a498e744524b8c253,govind dosas,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1627024612,False,Banjara Hills Road No 14,17.421322,78.434213,"[{'label': 'display', 'lat': 17.42132160823147...",7976,IN,India,"[Banjara Hills Road No 14, Hyderabad, Telangana]",Hyderabad,Telangana,NaN,NaN,NaN
2,59ec9b822632ec468f223fdd,Le Vantage,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1627024612,False,NaN,17.430074,78.417744,"[{'label': 'display', 'lat': 17.430074, 'lng':...",9778,IN,India,"[Hyderabad 500034, TG]",Hyderabad,TG,500034,NaN,NaN
3,5b8a9ca1336273002c2d675a,Aura,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1627024612,False,NaN,17.435822,78.412125,"[{'label': 'display', 'lat': 17.435822, 'lng':...",10650,IN,India,"[Hyderabad 500045, TG]",Hyderabad,TG,500045,NaN,NaN
4,5e42561c8fa40200081304be,Hush Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1627024612,False,"plot no 209, Kakatiya Hills, Madhapur, Hyderab...",17.442398,78.399460,"[{'label': 'display', 'lat': 17.442398, 'lng':...",12071,IN,India,"[plot no 209, Kakatiya Hills, Madhapur, Hydera...",Hyderabad,Telangana,500033,NaN,NaN


In [24]:
dataframe2.shape

(30, 18)

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list2 = row['categories']
    except:
        categories_list2 = row['venue.categories']
        
    if len(categories_list2) == 0:
        return None
    else:
        return categories_list2[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,city,state,postalCode,crossStreet,neighborhood,id
0,Hotel Nayaab,Food Court,Charminar,17.368504,78.477480,"[{'label': 'display', 'lat': 17.36850400859741...",953,IN,India,[Charminar],NaN,NaN,NaN,NaN,NaN,4f1db31ee4b04ae0856d18ff
1,govind dosas,Food Court,Banjara Hills Road No 14,17.421322,78.434213,"[{'label': 'display', 'lat': 17.42132160823147...",7976,IN,India,"[Banjara Hills Road No 14, Hyderabad, Telangana]",Hyderabad,Telangana,NaN,NaN,NaN,55c3a81a498e744524b8c253
2,Le Vantage,Italian Restaurant,NaN,17.430074,78.417744,"[{'label': 'display', 'lat': 17.430074, 'lng':...",9778,IN,India,"[Hyderabad 500034, TG]",Hyderabad,TG,500034,NaN,NaN,59ec9b822632ec468f223fdd
3,Aura,Café,NaN,17.435822,78.412125,"[{'label': 'display', 'lat': 17.435822, 'lng':...",10650,IN,India,"[Hyderabad 500045, TG]",Hyderabad,TG,500045,NaN,NaN,5b8a9ca1336273002c2d675a
4,Hush Cafe,Café,"plot no 209, Kakatiya Hills, Madhapur, Hyderab...",17.442398,78.399460,"[{'label': 'display', 'lat': 17.442398, 'lng':...",12071,IN,India,"[plot no 209, Kakatiya Hills, Madhapur, Hydera...",Hyderabad,Telangana,500033,NaN,NaN,5e42561c8fa40200081304be
5,Kritunga Restaurant,Indian Restaurant,NaN,17.438083,78.442897,"[{'label': 'display', 'lat': 17.43808273714060...",9239,IN,India,"[Ameerpet, Hyderabad, Telangana]",Hyderabad,Telangana,NaN,Ameerpet,NaN,4fec7952e4b028a3dc00a7b1
6,Blue Tokai,Coffee Shop,"Road Number 1, Nandagiri Hills",17.421579,78.410731,"[{'label': 'display', 'lat': 17.421579, 'lng':...",9558,IN,India,"[Road Number 1, Nandagiri Hills (Jubliee Hills...",Hyderabad,TG,500033,Jubliee Hills,NaN,5f7876c465c23a2096d03646
7,Al Marjan,Turkish Restaurant,paramount Hills gate number 2 tolichowki,17.407251,78.413410,"[{'label': 'display', 'lat': 17.407251, 'lng':...",8276,IN,India,"[paramount Hills gate number 2 tolichowki, Hyd...",Hyderabad,Telangana,500008,NaN,NaN,590318b3eacf4521f0a8a769
8,AnTeRa,South Indian Restaurant,Road No. 10,17.434596,78.414207,"[{'label': 'display', 'lat': 17.434596, 'lng':...",10406,IN,India,"[Road No. 10 (Jubilee Hills), Hyderabad 500045...",Hyderabad,TG,500045,Jubilee Hills,NaN,60eaa3ee9f7f843f60290202
9,Hole In The Wall,American Restaurant,NaN,17.424456,78.409680,"[{'label': 'display', 'lat': 17.424456, 'lng':...",9865,IN,India,"[Hyderabad 500033, Telangana]",Hyderabad,Telangana,500033,NaN,NaN,5aae5dbdbe7078047f609ce8


In [26]:
dataframe_filtered2.shape

(30, 16)

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel



for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# display map
venues_map

## Analysis <a name="analysis"></a>

We will identify all the restaurants in the city and all the famous places to visit in the city
We will create a distance matrix from all the restaurants to the famous places
We will compute the mean distance from restaurants to the famous place
We will identify the famous place with maximum average distance from any restaurant
The identified location will have a famous place but far away from any other restaurant

In [34]:
from scipy.spatial.distance import cdist
df_array_1 = dataframe_filtered[["lat", "lng"]].to_numpy()
df_array_2 = dataframe_filtered2[["lat", "lng"]].to_numpy()
dist_mat = cdist(df_array_1, df_array_2)
dist_mat
dist_table=pd.DataFrame(dist_mat)
dist_table

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.007493,0.072176,0.089086,0.097100,0.110423,0.082865,0.087704,0.076431,0.094824,0.090450,...,0.084569,0.087601,0.023232,0.037577,0.122267,0.095331,0.106544,0.081284,0.113677,0.080478
1,0.010913,0.072428,0.088946,0.096928,0.110055,0.083855,0.087097,0.075415,0.094686,0.089898,...,0.083802,0.088603,0.027016,0.039440,0.121653,0.095160,0.105693,0.084031,0.112793,0.079902
2,0.031011,0.044392,0.063035,0.070787,0.085032,0.049018,0.065875,0.059809,0.068371,0.067884,...,0.064541,0.053525,0.029278,0.065078,0.098435,0.069120,0.085595,0.045160,0.092641,0.059086
3,0.084089,0.044161,0.039355,0.042228,0.046401,0.061508,0.028546,0.019053,0.041835,0.030735,...,0.024186,0.063664,0.095807,0.121599,0.051063,0.041347,0.033968,0.100402,0.039113,0.027856
4,0.066628,0.017858,0.032460,0.038626,0.052401,0.011428,0.040210,0.042881,0.036377,0.040802,...,0.041586,0.015941,0.066830,0.102634,0.066340,0.037304,0.057536,0.039223,0.063725,0.036012
5,0.056935,0.021027,0.038595,0.045616,0.059771,0.021143,0.044510,0.044001,0.043250,0.045693,...,0.044874,0.025711,0.057156,0.092878,0.073610,0.044116,0.063171,0.038336,0.069763,0.039088
6,0.068100,0.105939,0.123952,0.130871,0.144884,0.102462,0.129064,0.124760,0.128534,0.130612,...,0.128501,0.105580,0.051315,0.062680,0.158798,0.129417,0.148300,0.065767,0.155029,0.122811
7,0.036248,0.039335,0.057981,0.065643,0.079908,0.043325,0.061343,0.056150,0.063227,0.063228,...,0.060282,0.047825,0.034946,0.070780,0.093418,0.063998,0.080986,0.041995,0.087973,0.054715
8,0.005782,0.073174,0.090906,0.098942,0.112678,0.081803,0.090693,0.080501,0.096597,0.093284,...,0.087972,0.086467,0.013651,0.033571,0.125078,0.097184,0.110011,0.075186,0.117190,0.083454
9,0.018385,0.081662,0.097924,0.105874,0.118826,0.093289,0.095673,0.083561,0.103660,0.098524,...,0.092217,0.098037,0.031883,0.034112,0.130176,0.104109,0.113971,0.092383,0.121020,0.088523


Let's identify the place which is far away from the existing restaurants

identify the maximum of mean distance from each restaurant

In [35]:
minValues = dist_mat.mean(axis = 1)
print(minValues)
maxvalue=minValues.max()
maxvalue

[0.08337052 0.08357812 0.06247475 0.04912463 0.04281808 0.04571977
 0.11798947 0.05873616 0.08439644 0.09172343 0.03293169 0.06663454
 0.04507374 0.03941414 0.06646716 0.04281786 0.14652737 0.06365807
 0.11712595 0.11507544 0.11752504 0.05457248 0.06066687 0.04411565
 0.04041043 0.04557769 0.11083863 0.04448996 0.04277839 0.04421664]


0.1465273708488436

## Result <a name="result"></a>

In [38]:
result = np.where(minValues == 0.1465273708488436)
print(result)

(array([16], dtype=int64),)


In [58]:
dataframe_filtered.name[16]

'KK Gardens'

Let's mark the result on map with green colour which is the ideal place to start a restaurant

In [54]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# All restaurants marked in red colour
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    folium.CircleMarker(
        [dataframe_filtered.iloc[16]['lat'], dataframe_filtered.iloc[16]['lng']],
        radius=50,
        color='green',
        popup=dataframe_filtered.iloc[16]['name'],
        fill = True,
        fill_color='green',
        fill_opacity=0.01
    ).add_to(venues_map)

# display map
venues_map

## Discussion

FOursquare API returns only 30 famous places and 30 restaurants for a request. So the project is built accordingly

Also, we are assuming that all the famous places are inhabitable and easy to start a new restaurant

We can further improve the model by identifying the ideal location which is close to multiple famous places and also having least number of restaurants

## Conclusion <a name="conclusion"></a>

Using this program, we can identify a famous place which is having no/less restaurants in its vicinity
So that we can setup a restaurant in the area nearby